In [9]:
# 3 Customer Journey & Engagement Analysis (SQL):
# Identify drop-off points in the customer journey.

import pandas as pd
import pymysql
import mysql.connector as db
from datetime import datetime
from sqlalchemy import create_engine

user  = 'root'
passcode = "password12*"
host = 'localhost'
db_name = "shopeasy"

db_connection1 = db.connect(
                host  = host,
                user = user,
                password = passcode,
                database = db_name
)

db_connection1
curr = db_connection1.cursor()

sql1 = """
                 SELECT Stage, COUNT(*) AS DropOffCount
                        FROM shopeasy.customer_journey
                 WHERE Action = 'Drop-off' 
                        GROUP BY Stage
                        ORDER BY DropOffCount DESC;

     """

sql2 =  """
                WITH LastInteraction AS (
                    SELECT CustomerID, ProductID, VisitDate, Stage, Action,
                                   ROW_NUMBER() OVER (PARTITION BY CustomerID ORDER BY VisitDate DESC) AS rn
                    FROM shopeasy.customer_journey
                                    WHERE Action = 'Drop-off'
                                      )
                    SELECT CustomerID, ProductID, VisitDate, Stage FROM LastInteraction WHERE rn = 1;
      """      

# Find common actions leading to successful conversions.

sql3 =  """
                    SELECT Action, COUNT(*) AS ActionCount FROM shopeasy.customer_journey
                            WHERE CustomerID IN (
                   
                    SELECT DISTINCT CustomerID FROM shopeasy.customer_journey 
                            WHERE Stage = 'Checkout' AND Action = 'Purchase'
                                                )
                    GROUP BY Action
                            ORDER BY ActionCount DESC;
      """     


sql4 =  """
                    SELECT Stage, AVG(Duration) AS AvgDuration
                        FROM shopeasy.customer_journey
                    WHERE Duration IS NOT NULL GROUP BY Stage
                       ORDER BY AvgDuration DESC;
      """   
curr.execute(sql1)
results1 = curr.fetchall()

curr.execute(sql2)
results2 = curr.fetchall()

curr.execute(sql3)
results3 = curr.fetchall()

curr.execute(sql4)
results4 = curr.fetchall()

# common actions leading to successful conversions - Process results
#print("\n",results1)

#Identify drop-off points in the customer journey
#print("\n",results2)

#Identify drop-off points in the customer journey
#print("\n",results3)

df1 = pd.read_sql_query(sql1, db_connection1)
print(df1.head())
print("\n\n")
df2 = pd.read_sql_query(sql2, db_connection1)
print(df2.head())

print("\n Common actions leading to successful conversions - Process results \n")

df3 = pd.read_sql_query(sql3, db_connection1)
print(df3.head())

print("\n Calculate average duration per stage for engagement insights. \n")

df4 = pd.read_sql_query(sql4, db_connection1)
print(df4.head())



      Stage  DropOffCount
0  Checkout            14



   CustomerID  ProductID   VisitDate     Stage
0           1          7  2024-03-17  Checkout
1           9         11  2024-11-19  checkout
2          15          9  2025-08-07  Checkout
3          23         12  2023-08-22  Checkout
4          30         20  2024-02-18  checkout

 Common actions leading to successful conversions - Process results 

     Action  ActionCount
0  Purchase            6
1      View            2
2  Drop-off            1

 Calculate average duration per stage for engagement insights. 

         Stage  AvgDuration
0  ProductPage     182.7692
1     Homepage     160.5370
2     Checkout      45.0500


C:\Users\I02062\AppData\Local\Temp\ipykernel_1038480\2590357080.py:85: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql_query(sql1, db_connection1)
C:\Users\I02062\AppData\Local\Temp\ipykernel_1038480\2590357080.py:88: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql_query(sql2, db_connection1)
C:\Users\I02062\AppData\Local\Temp\ipykernel_1038480\2590357080.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df3 = pd.read_sql_query(sql3, db_connection1)
C:\Users\I02062\AppData\Local\Temp\ipyke